In [1]:
import numpy as np
import cv2 # OpenCV biblioteka
import matplotlib
import matplotlib.pyplot as plt
import os, sys

def process_video(vidcap, sec, i, count):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)            # postavi na svakih 4 sekunde da uzima snimak
    hasFrames,image = vidcap.read()                       # procitaj frejm
    if hasFrames:                                         # ako ima i dalje snimaka
        img_crop = image[145:430, 265:390]                # iseci deo platoa, prva koordinata je po visini (formalno red), druga po širini (formalo kolona)
        cv2.imwrite('image'+str(count)+'.jpg', img_crop)  # sacuvaj frejm kao JPG fajl
    return hasFrames

def load_image(path):
    # ucitavanje slike i konvertovanje u RGB
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def image_gray(image):
    # konvert u grayscale
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def image_bin(image_gs):
    # binarna slika
    ret, image_bin = cv2.threshold(image_gs, 52, 255, cv2.THRESH_BINARY)
    return image_bin

def erode(image):
    # umanjivanje belih piksela, onih koji se pojavljuju kao visak
    kernel = np.ones((2, 2)) # strukturni element 2x2 blok
    return cv2.erode(image, kernel, iterations=1)

def dilate(image):
    # uvecavanje belih piksela, dobijanje vece bele povrsine, koja se kasnije koristi za konturu
    kernel = np.ones((2, 2)) # strukturni element 2x2 blok
    return cv2.dilate(image, kernel, iterations=4)

def select_roi(image):
    # izdvajanje kontura(regiona od interesea)
    image, contours, hierarchy = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def sum_all_contours(contours):    
    # lista za konture
    contours_people = []
    
    # za svaku konturu
    for contour in contours:
        # pronadji "pravougaonik" minimalne povrsine koji ce obuhvatiti celu konturu
        center, size, angle = cv2.minAreaRect(contour)
        width, height = size
        if width >= 2 and height >= 2:              # uslov da konturu ubacimo u listu svih kontura
            contours_people.append(contour)
    return len(contours_people)                     # vracamo velicinu liste, tj. broj kontura koji se nalazi u listi

realValues = [4, 24, 17, 23, 17, 27, 29, 22, 10, 23]   # tacne vrednosti
finalSum = 0                                           # suma za racunanje mae
for i in range(1, 11, 1):                              # za svaki video
    direct = 'picturesVideo'+str(i)+'/'
    os.mkdir(direct)
    os.chdir(direct)
    people = 0
    sec = 0
    frameRate = 4                                      # belezice snimak svake 4 sekunde
    count = 1
    currentVideo = '../data/video'+str(i)+'.mp4'
    vidcap = cv2.VideoCapture(currentVideo)            # ucitavanje trenutnog videa za procesiranje
    success = process_video(vidcap, sec, i, count)     # pravljenje slika za trenutni video koji obradjujemo
    while success:                                     # dok ima frejmova pravi slike i smestaj ih u trenutni folder
        count = count + 1
        sec = sec + frameRate
        success = process_video(vidcap, sec, i, count)
    for j in range(1, len([name for name in os.listdir('.')]) + 1, 1): # za svaku sliku u trenutnom folderu
        loadImg = load_image('image'+str(j)+'.jpg')
        imageGray = image_gray(loadImg)
        imageBin = image_bin(imageGray)
        er = erode(imageBin)
        di = dilate(er)
        contours = select_roi(di)
        detected = sum_all_contours(contours)
        if detected is None:
            detected = 0
        people += detected
    finalSum += np.absolute(people-realValues[i-1])
    print(str(i)+'. video', ' detected: ' , people, ' people;', 'real value: ', realValues[i-1])
    os.chdir('..')
    
print()
print('Mean absolute error: ', finalSum/10)


1. video  detected:  4  people; real value:  4
2. video  detected:  25  people; real value:  24
3. video  detected:  17  people; real value:  17
4. video  detected:  26  people; real value:  23
5. video  detected:  21  people; real value:  17
6. video  detected:  28  people; real value:  27
7. video  detected:  28  people; real value:  29
8. video  detected:  19  people; real value:  22
9. video  detected:  11  people; real value:  10
10. video  detected:  26  people; real value:  23

Mean absolute error:  1.7
